___
## Ciência dos Dados - PROJETO 3 - INSPER

___
#### Mariana Abrantes

#### Gabriel Couto

#### Gabriel Miras
___

# Previsão de preços de smartphones ao dólar

___

## A. INTRODUÇÃO

Nestes últimos anos, com a instabilidade política e socioeconômica do Brasil, o preço do dólar está em constante variação, e com isso o preço de produtos estrangeiros mudam constantemente. Movidos por uma curiosidade de encontrar se estes fatores possuem semelhanças, escolhemos criar um modelo preditivo que, baseados em diversos fatores acerca de um modelo de smartphone, faça uma previsão acerca de seu preço durante os meses. Estamos interessados em buscar modelos consistentes, que nos permitam observar as predições e tirar conclusões acerca de sua performance.

___
## B. MINERANDO DADOS e CARACTERÍSTICAS DO DATASET

[Detalhe aqui as características da base de dados além da análise descritiva.]

In [1]:
import pandas as pd
import requests
import os
import re

#Preço do dolar
dolar19 = pd.read_csv('dolar 19.csv')
dolar18 = pd.read_csv('dolar 18.csv')
dolar   = dolar19.append(dolar18)
dolar['Data'] = dolar['Data'].str.replace('.','/', regex=False)
dolar.index = dolar['Data']
dol = pd.DataFrame(columns = dolar.index)
dol = dol.append(dolar.T.loc['Último',:])

In [2]:
#Recebe uma URL e retorna o código HTML da página (Só é chamada pela função baixarBD)
def baixarhtml(url):
    html = requests.get(url).text.splitlines()
    return html

#Recebe url do Buscape e retorna um Dataframe 
def baixarBD(url):
    #Pegar ID do produto do buscape usando a url fornecida
    code = baixarhtml(url)
    for i in range(len(code)):
        igual = re.findall('"productId":[0-9]*', code[i])
        if len(igual)>0:
            id = igual[0].split(':')[1]
            
    #Baixar BD dos preços
    site = "https://www.buscape.com.br/async/history/idu/" + id + "/days/365"
    code = baixarhtml(site)
    
    #Converte código e cria tabela
    c = code[0].split('"historicos":')
    json_file = c[1][:-1]
    bd = pd.read_json(json_file)
    return bd.loc[:,['data','precomed','precomin']]

    
#Recebe lista com o nome do produto e links e baixa todos eles
def df_min(lista):
    for num in range(len(lista)):
        produto, link = lista[num][0], lista[num][1]
        df0 = baixarBD(link)
        df = df0.loc[:,['precomin']].T #Cria Tabela de precomin por data
        df.index = [produto]
        if num == 0:
            bd = pd.DataFrame(columns = range(len(df)))
            bd = bd.append(df)
        else:
            bd = bd.append(df)
            
    bd.columns = list(df0['data'])
    return bd

### BASE DE DADOS

In [8]:
dol

Data,09/05/2019,08/05/2019,07/05/2019,06/05/2019,05/05/2019,03/05/2019,02/05/2019,01/05/2019,30/04/2019,29/04/2019,...,12/01/2018,11/01/2018,10/01/2018,09/01/2018,08/01/2018,05/01/2018,04/01/2018,03/01/2018,02/01/2018,01/01/2018
Último,"3,9420","3,9285","3,9710","3,9695","3,9383","3,9382","3,9661","3,9202","3,9215","3,9456",...,"3,2064","3,2155","3,2283","3,2483","3,2407","3,2292","3,2346","3,2370","3,2591","3,3125"


In [3]:
links = [
    ['Iphone X',          "https://www.buscape.com.br/smartphone-apple-iphone-x-64gb"],
    ['Iphone 8 Plus',     "https://www.buscape.com.br/smartphone-apple-iphone-8-plus-64gb"],
    ['Iphone 7',          "https://www.buscape.com.br/smartphone-apple-iphone-7-32gb"],
    ['Iphone 7 Plus',     "https://www.buscape.com.br/smartphone-apple-iphone-7-plus-128gb"],
    ['Galaxy S8',         "https://www.buscape.com.br/smartphone-samsung-galaxy-s8-sm-g950-64gb"],
    ['Galaxy S8 Plus',    "https://www.buscape.com.br/smartphone-samsung-galaxy-s8-plus-sm-g955-128gb"],
    ['Galaxy S9',         "https://www.buscape.com.br/smartphone-samsung-galaxy-s9-sm-g960-128gb"],
    ['Galaxy S9 Plus',    "https://www.buscape.com.br/smartphone-samsung-galaxy-s9-plus-sm-g965-128gb"]
]

In [4]:
dfProdutos = df_min(links)

In [5]:
dfProdutos.head()

,18/05/2018,19/05/2018,20/05/2018,21/05/2018,22/05/2018,23/05/2018,24/05/2018,25/05/2018,26/05/2018,27/05/2018,...,07/05/2019,08/05/2019,09/05/2019,10/05/2019,11/05/2019,12/05/2019,13/05/2019,14/05/2019,15/05/2019,16/05/2019
Iphone X,4998.96,4998.96,4998.96,4998.96,5099.00,5099.00,5099.00,5219.10,5099.00,5099.0,...,4599.00,4199.00,4399.00,4199.00,4399.00,4199.00,4399.00,4199.00,4399.00,4249.00
Iphone 8 Plus,3399.00,3518.24,3599.09,3554.10,3554.10,3475.12,3484.15,3484.15,3519.11,3509.1,...,3144.15,3239.10,3239.10,3239.10,3239.10,3239.10,3239.10,3329.10,3144.15,3329.10
Iphone 7,2519.10,2519.10,2519.10,2420.10,2449.79,2375.10,2375.10,2375.10,2375.10,2375.1,...,2294.15,2294.15,2284.32,2284.32,2284.32,2284.32,2284.32,2349.21,2349.21,2349.21
Iphone 7 Plus,3239.10,3239.10,3239.10,3239.10,3132.90,3089.00,3089.00,3059.00,3059.00,3059.0,...,3059.15,3059.15,3059.15,3167.12,3167.12,3167.12,3167.12,3167.12,3167.12,3123.12
Galaxy S8,2429.19,2474.10,2479.90,2499.00,2519.99,2519.99,2429.19,2519.99,2479.90,2479.9,...,2599.00,2114.94,2729.00,2649.99,2649.99,2649.99,2649.99,2649.00,2729.00,2543.30


### ANÁLISE DESCRITIVA

___
## C. MODELOS DE PREDIÇÃO

o MODELO DE PREDIÇÃO PELA MÉDIA (Sem uso de variável explicativa).

o MODELO DOS K VIZINHOS MAIS PRÓXIMOS (K-Nearest Neighbors Regression)

o MODELO DE REGRESSÃO LINEAR (Multiple Linear Regression)

o MODELO DE ÁRVORES DE REGRESSÃO (Decision Tree Regression)

___
## D. PROCESSO E ESTATÍSTICAS DE VALIDAÇÃO

[Esse item depende dos resultados das modelagens anteriores! Organize-os aqui de forma clara!]

___
## E. CONCLUSÃO

___
## F. REFERÊNCIAS BIBLIOGRÁFICAS